In [1]:
import mlflow
import torch
import numpy as np
from utils.dataImportation import import_data, import_earnings_date, merge_options_earnings, \
    apply_filters_to_options, generate_tte, split_in_sets, get_dataloader, get_datasets
from utils.staticModels import SplitFeedforwardNNList, SplitDeepFactorNNList, SplitFrancois

/Users/sebastienlegros/Developer/DIVFM/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
path_to_database = '../data'
STOCK = 'AAPL'
USE_MLFLOW = True
USE_EARNINGS = True

BEGIN_TRAIN_DATE = '2010-01-01'  # '2000-01-01'
END_TRAIN_DATE = '2011-01-01'  # '2021-01-01'
BEGIN_TEST_DATE = '2011-06-01' # '2022-01-01'
FEATURES_NAME = ['ttm', 'ttm_scaled_moneyness', 'time_to_earnings'] if USE_EARNINGS else ['ttm', 'ttm_scaled_moneyness']

DEVICE = 'cpu'
 

USE_LOG_VOL = True
USE_ARBITRAGE_PENALTY = False
USE_FRANCOIS_ET_AL = False

ADD_LEVEL_AS_FACTOR = True
ADD_TTM_SLOPE = False
NUM_FACTORS = 4
INPUT_SHAPE = 2
INPUT_TTM = 2
NUM_NEURONS = 64
NUM_LAYERS = 3
ACTIVATION_FUNCTION = torch.nn.GELU
DROPOUT = 0.0
OUTPUT_ACTIVATION = None
BATCH_NORM = True
OUT_BATCH_NORM = True


BATCH_SIZE = 32
LR = 5e-4
EPOCHS = 200
SCHEDULER_STEP_SIZE = 50
SCHEDULER_GAMMA = 0.1
CLIPPING = 0.1



In [3]:
if STOCK == 'SPX':
    USE_EARNINGS = False

In [4]:
labels_name = ['LOG_OM_IV'] if USE_LOG_VOL else ['OM_IV']
model_output_fn = torch.exp if USE_LOG_VOL else torch.nn.Identity()

In [5]:
options_data = import_data(path_to_database, STOCK)
earning_dates = import_earnings_date(path_to_database, STOCK)
options_data = merge_options_earnings(options_data, earning_dates)
options_data = apply_filters_to_options(options_data)
if USE_EARNINGS: options_data = generate_tte(options_data, earning_dates)

train_data, valid_data, test_data = split_in_sets(options_data, BEGIN_TRAIN_DATE, END_TRAIN_DATE, BEGIN_TEST_DATE)
train_dataset, valid_dataset, test_dataset = get_datasets(train_data, valid_data, test_data,
                                                          FEATURES_NAME, labels_name, dtype=torch.float32)
train_loader, valid_loader, test_loader = get_dataloader(train_dataset, valid_dataset, test_dataset, BATCH_SIZE)

In [6]:
if USE_EARNINGS:
    input_size = [1, 2, 2]
    factors_inputs_index = [[1], [0, 2], [0, 1]]
else:
    input_size = [1, 1, 2]
    factors_inputs_index = [[1], [0], [0, 1]]
    
    
split_feedforward = SplitFeedforwardNNList(input_size=input_size, output_size=[1, 1, 2],
                                           num_neurons=NUM_NEURONS, num_layers=NUM_LAYERS, activation_fn=ACTIVATION_FUNCTION,
                                           dropout=DROPOUT, output_activation=OUTPUT_ACTIVATION, batch_norm=BATCH_NORM)

# ['ttm', 'ttm_scaled_moneyness', 'time_to_earnings']

model = SplitDeepFactorNNList(splitFeedforwardNNList=split_feedforward, 
                      factors_inputs_index=factors_inputs_index,
                      out_batch_norm=OUT_BATCH_NORM,
                      output_activation=OUTPUT_ACTIVATION)
if USE_FRANCOIS_ET_AL:
    model = SplitFrancois(1, 0)


In [7]:
optimizer = torch.optim.Adam(list(model.parameters()), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=SCHEDULER_STEP_SIZE, gamma=SCHEDULER_GAMMA)

In [8]:
mlflow.end_run()

In [9]:
if USE_EARNINGS:
    mlflow.set_experiment(f'{STOCK} Log Penalty Factor Model')
else:
    mlflow.set_experiment(f'{STOCK} Log Penalty Factor Model - without earnings')
mlflow.start_run()
run_id = mlflow.active_run().info.run_id

mlflow.set_tag('arbitrage_penalty', USE_ARBITRAGE_PENALTY)

In [10]:
mlflow.active_run().info.artifact_uri

'file:///Users/sebastienlegros/Developer/DIVFM/notebooks/mlruns/452981282717470609/0eb20054cba64168bedf9eacfb8e6d46/artifacts'

In [11]:
params = dict(
    # Dataset
    STOCK=STOCK,
    USE_EARNINGS=USE_EARNINGS,
    BEGIN_TRAIN_DATE=BEGIN_TRAIN_DATE,
    END_TRAIN_DATE=END_TRAIN_DATE,
    BEGIN_TEST_DATE=BEGIN_TEST_DATE,
    FEATURES_NAME=FEATURES_NAME,

    # Device
    DEVICE=DEVICE,

    # Model switches
    USE_FRANCOIS_ET_AL=USE_FRANCOIS_ET_AL,
    USE_LOG_VOL=USE_LOG_VOL,
    USE_ARBITRAGE_PENALTY=USE_ARBITRAGE_PENALTY,
    ADD_LEVEL_AS_FACTOR=ADD_LEVEL_AS_FACTOR,
    ADD_TTM_SLOPE=ADD_TTM_SLOPE,

    # Architecture
    NUM_FACTORS=NUM_FACTORS,
    INPUT_SHAPE=INPUT_SHAPE,
    INPUT_TTM=INPUT_TTM,
    NUM_NEURONS=NUM_NEURONS,
    NUM_LAYERS=NUM_LAYERS,
    ACTIVATION_FUNCTION=ACTIVATION_FUNCTION,
    DROPOUT=DROPOUT,
    OUTPUT_ACTIVATION=OUTPUT_ACTIVATION,
    BATCH_NORM=BATCH_NORM,
    OUT_BATCH_NORM=OUT_BATCH_NORM,

    # Training
    BATCH_SIZE=BATCH_SIZE,
    LR=LR,
    EPOCHS=EPOCHS,
    SCHEDULER_STEP_SIZE=SCHEDULER_STEP_SIZE,
    SCHEDULER_GAMMA=SCHEDULER_GAMMA,
    CLIPPING=CLIPPING,
)
mlflow.log_params(params)

In [12]:
device = DEVICE
loss_fn = torch.nn.MSELoss()
m_col = 1
tau_col = 0
penalty_weight = 1.
penalty = torch.tensor(0.)
model.to(DEVICE)
for epoch in range(EPOCHS):
    model.train()
    train_loss_sum, train_fit_sum, train_penalty_sum, n_batches = 0.0, 0.0, 0.0, 0
    for features, labels, num_obs_per_group, _ in train_loader:
        n_batches += 1 
        features, labels = features.to(device), labels.to(device)
        factors, betas, logpred = model(features, labels, num_obs_per_group)
        fitting_loss = loss_fn(torch.exp(logpred), torch.exp(labels))
        
        # # penalty graph (inputs need grads here only)
        if USE_ARBITRAGE_PENALTY:
            x = features.detach().clone().requires_grad_(True)
            _, _, log_IV_pred_p = model(x, labels, num_obs_per_group)


            # compute derivatives w.r.t. M on IV_pred_p
            n = log_IV_pred_p if log_IV_pred_p.ndim == 2 else log_IV_pred_p.unsqueeze(1)

            n_M_full = torch.autograd.grad(
                outputs=n, inputs=x, grad_outputs=torch.ones_like(n),
                create_graph=True, retain_graph=True
            )[0]
            n_M = n_M_full[:, m_col:m_col + 1]

            n_MM_full = torch.autograd.grad(
                outputs=n_M, inputs=x, grad_outputs=torch.ones_like(n_M),
                create_graph=True
            )[0]
            n_MM = n_MM_full[:, m_col:m_col + 1]

            M = x[:, m_col:m_col + 1]
            tau = x[:, tau_col:tau_col + 1]

            expr = n_MM + n_M ** 2 + torch.exp(2 * n) * (1 - n_M * M) ** 2 \
                   - (tau / 4.0) * n_M ** 2 * torch.exp(2 * n)
            penalty = torch.relu(-expr).mean()
        
        # total loss
        loss = fitting_loss + penalty_weight * penalty
        
        optimizer.zero_grad()
        loss.backward()
        if CLIPPING is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CLIPPING)
        optimizer.step()
        
        train_loss_sum += loss.detach().item()
        train_fit_sum += fitting_loss.detach().item()
        train_penalty_sum += penalty.detach().item()

    train_total = train_loss_sum / n_batches
    train_rmse = float(np.sqrt(train_fit_sum / n_batches))
    train_penalty = train_penalty_sum / n_batches

    if scheduler is not None:
        scheduler.step()

    # Validation (no penalty)
    valid_rmse = None
    if valid_loader is not None:
        model.eval()
        with torch.no_grad():
            v_sum, v_batches = 0.0, 0
            for features, labels, num_obs_per_group, groups in valid_loader:
                v_batches += 1
                features, labels = features.to(device), labels.to(device)
                factors, betas, logpred = model(features, labels, num_obs_per_group)
                fitting_loss = loss_fn(torch.exp(logpred), torch.exp(labels))
                v_sum += fitting_loss.item()
            valid_rmse = float(np.sqrt(v_sum / v_batches))

    
    print(f"epoch {epoch:03d} | train total {train_total:.5f} | "
          f"train RMSE {train_rmse:.5f} | valid RMSE {valid_rmse:.5f}")

    if USE_MLFLOW:
        mlflow.log_metric('train_RMSE', train_rmse, step=epoch)
        mlflow.log_metric('train_total_loss', train_total, step=epoch)
        mlflow.log_metric('valid_RMSE', valid_rmse, step=epoch)
        mlflow.log_metric('train_penalty', train_penalty, step=epoch)


    # if valid_rmse is not None and (best_val is None or valid_rmse < best_val):
    #     best_val = valid_rmse
    #     best_model = copy.copy(model)

epoch 000 | train total 0.00020 | train RMSE 0.01423 | valid RMSE 0.00291
epoch 001 | train total 0.00010 | train RMSE 0.00998 | valid RMSE 0.00303
epoch 002 | train total 0.00008 | train RMSE 0.00886 | valid RMSE 0.00248
epoch 003 | train total 0.00008 | train RMSE 0.00914 | valid RMSE 0.00279
epoch 004 | train total 0.00007 | train RMSE 0.00859 | valid RMSE 0.00315
epoch 005 | train total 0.00007 | train RMSE 0.00841 | valid RMSE 0.00238
epoch 006 | train total 0.00007 | train RMSE 0.00853 | valid RMSE 0.00238
epoch 007 | train total 0.00006 | train RMSE 0.00789 | valid RMSE 0.00247
epoch 008 | train total 0.00006 | train RMSE 0.00762 | valid RMSE 0.00232
epoch 009 | train total 0.00006 | train RMSE 0.00805 | valid RMSE 0.00337
epoch 010 | train total 0.00007 | train RMSE 0.00864 | valid RMSE 0.00327
epoch 011 | train total 0.00008 | train RMSE 0.00918 | valid RMSE 0.00337
epoch 012 | train total 0.00008 | train RMSE 0.00901 | valid RMSE 0.00330
epoch 013 | train total 0.00008 | trai

KeyboardInterrupt: 

In [13]:
mlflow.pytorch.log_model(model.to('cpu'), 'model')

2026/01/11 16:13:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/11 16:13:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [14]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n_params = count_trainable_parameters(model)
print(f"Trainable parameters: {n_params}")

Trainable parameters: 39364
